In [ ]:
"""This notebook is the summed up version of all the various methods that I applies to fill the NaN's. In various outputs sometimes I just used median,
sometime didn't factor for outliers,even forgot to drop some rows.Now since i continuosly made changes  in the same notebook i couldn't save them for the
link submission. But I hope this notebook would be enough has I have described all the various methods that I used."""

In [75]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [76]:
data = pd.read_csv('D:/downloads/hacktrain.csv')
data

,Unnamed: 0,ID,class,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,water,637.5950,658.668,-1882.030,-1924.36,997.904,-1739.990,630.087,...,NaN,-1043.160,-1942.490,267.138,NaN,NaN,211.328,-2203.020,-1180.19,433.906
1,1,2,water,634.2400,593.705,-1625.790,-1672.32,914.198,-692.386,707.626,...,NaN,-933.934,-625.385,120.059,364.858,476.972,220.878,-2250.000,-1360.56,524.075
2,3,4,water,58.0174,-1599.160,NaN,-1052.63,NaN,-1564.630,NaN,...,-1025.880,368.622,NaN,-1227.800,304.621,NaN,369.214,-2202.120,NaN,-1343.550
3,4,5,water,72.5180,NaN,380.436,-1256.93,515.805,-1413.180,-802.942,...,-1813.950,155.624,NaN,-924.073,432.150,282.833,298.320,-2197.360,NaN,-826.727
4,7,8,water,1136.4400,NaN,NaN,1647.83,1935.800,NaN,2158.980,...,1535.000,1959.430,-279.317,-384.915,-113.406,1020.720,1660.650,-116.801,-568.05,-1357.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,10537,10538,impervious,1207.7000,984.620,NaN,1166.25,937.478,1072.700,823.896,...,1117.740,1176.600,1044.110,NaN,369.082,465.843,362.882,979.795,NaN,433.659
7996,10538,10539,impervious,2170.3500,1419.720,1361.000,1478.71,983.911,1262.110,1422.860,...,984.634,2128.970,1379.660,NaN,762.633,485.204,446.724,771.747,1589.06,506.936
7997,10541,10542,impervious,1895.6800,1454.740,NaN,1033.56,1930.380,1057.150,1471.600,...,888.408,2093.020,1232.110,1190.830,1441.460,1170.880,1095.000,1818.650,2501.72,1247.770
7998,10542,10543,impervious,3465.7400,1283.320,413.412,4391.05,1146.820,4473.050,1614.750,...,5833.760,4047.320,4515.800,433.177,277.296,744.143,NaN,3759.710,NaN,388.346


In [77]:
df = data.copy()

In [78]:
df['class'].unique()

array(['water', 'forest', 'impervious', 'farm', 'grass', 'orchard'],
      dtype=object)

In [79]:
df = df[df.isna().sum(axis=1) < 8]
df.shape

(7932, 30)

In [80]:
def func1(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series[(series >= lower) & (series <= upper)]

def func2(df, col, drop_col):
    result = {}
    drop_col_names = df.columns[drop_col]
    value_cols = df.columns.drop([col] + list(drop_col_names))

    for class_val, group in df.groupby(col):
        group_result = {}
        for val_col in value_cols:
            col_data = group[val_col].dropna()
            cleaned = func1(col_data)
            if not cleaned.empty:
                group_result[val_col] = {'mean': cleaned.mean(), 'median': cleaned.median()}
        if group_result:
            result[class_val] = group_result

    return result

a = func2(df, 'class', [0,1])
a

{'farm': {'20150720_N': {'mean': 4837.837929102564,
   'median': 5075.0650000000005},
  '20150602_N': {'mean': 4085.501350069735, 'median': 4113.81},
  '20150517_N': {'mean': 3325.977235006605, 'median': 3248.5},
  '20150501_N': {'mean': 4293.61682946794, 'median': 4234.21},
  '20150415_N': {'mean': 1829.573678618858, 'median': 1154.29},
  '20150330_N': {'mean': 5190.428781073447, 'median': 5680.205},
  '20150314_N': {'mean': 4374.347878381963, 'median': 4786.97},
  '20150226_N': {'mean': 5086.373640287769, 'median': 5690.54},
  '20150210_N': {'mean': 3433.2643718791064, 'median': 3084.26},
  '20150125_N': {'mean': 3356.168578875171, 'median': 3107.47},
  '20150109_N': {'mean': 1245.271463418291, 'median': 1000.31},
  '20141117_N': {'mean': 2670.8669162995598, 'median': 2432.31},
  '20141101_N': {'mean': 2526.80548725, 'median': 2204.67},
  '20141016_N': {'mean': 1716.8440677710846, 'median': 1166.595},
  '20140930_N': {'mean': 2311.185981771812, 'median': 1341.58},
  '20140813_N': {'m

In [81]:
b = func2(df, 'class', [0, 1])

for col in df.columns:
    if col in df.columns[[0, 1]] or col == 'class':
        continue
    df[col] = df.apply(lambda row: ((b[row['class']][col]['median'] + b[row['class']][col]['mean']) / 2)  if pd.isna(row[col]) else row[col], axis=1)


In [82]:
df.isna().sum()

Unnamed: 0    0
ID            0
class         0
20150720_N    0
20150602_N    0
20150517_N    0
20150501_N    0
20150415_N    0
20150330_N    0
20150314_N    0
20150226_N    0
20150210_N    0
20150125_N    0
20150109_N    0
20141117_N    0
20141101_N    0
20141016_N    0
20140930_N    0
20140813_N    0
20140626_N    0
20140610_N    0
20140525_N    0
20140509_N    0
20140423_N    0
20140407_N    0
20140322_N    0
20140218_N    0
20140202_N    0
20140117_N    0
20140101_N    0
dtype: int64

In [83]:
test = pd.read_csv('D:/downloads/hacktest.csv')
test

,Unnamed: 0,ID,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,20150226_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,7466.4200,413.162,5761.000,5625.45,489.4030,3923.84,3097.110,6766.42000,...,801.184,927.115,4704.14,6378.42,340.949,2695.5700,527.268,4736.75,601.843,6639.760
1,1,2,7235.2600,6037.350,1027.560,6085.14,1618.0500,6668.54,2513.990,1051.69000,...,5533.470,5103.040,5216.12,4885.27,4366.790,1234.1400,3298.110,6942.68,1070.440,842.101
2,2,3,7425.0800,6969.980,1177.940,7408.93,861.0610,7644.43,814.458,1504.29000,...,1981.390,6204.540,7021.69,5704.41,4897.450,1789.9900,2206.100,6928.93,1036.560,831.441
3,3,4,7119.1200,1731.620,6311.930,6441.61,465.9790,7128.42,1649.120,6935.22000,...,959.344,5794.150,1045.57,5572.90,586.287,685.9060,1287.000,6734.72,824.584,6883.610
4,4,5,7519.5500,8130.260,1482.540,7879.53,1001.2100,7937.60,4122.530,1094.51000,...,7636.070,6996.760,7413.43,4596.13,4511.700,1413.5200,3283.940,7937.68,1857.800,1336.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840,2840,2841,-1673.7400,-2514.480,-2451.190,-2738.44,64.4464,-2275.03,-2881.100,-4738.97000,...,-2257.890,-2582.420,-1699.10,-2295.30,306.483,209.4030,221.920,-4885.15,-2630.590,356.233
2841,2841,2842,-96.8233,-412.727,-1795.400,-2363.82,-2168.1900,-2162.68,-3155.740,-4416.11000,...,-3991.910,-2614.910,-2109.84,-2687.18,-2853.890,71.3402,-86.414,-5109.01,324.637,-4316.580
2842,2842,2843,-2364.6000,-155.592,-1422.090,-1713.40,465.6220,-2230.40,-3088.730,-5010.32000,...,-2484.500,-1756.080,-3820.43,-1988.23,433.852,291.8000,254.548,-4259.30,412.115,-1170.750
2843,2843,2844,-3004.6300,-1217.120,180.122,-1113.89,438.4180,-2442.51,-3210.560,-3237.74000,...,-3291.490,-2018.450,-3472.65,-3109.07,511.792,369.0970,276.948,-5574.51,298.739,460.419


In [84]:
X_test = test.iloc[:,2:29]

In [85]:
X_train = df.iloc[:,3:30]
y_train = df['class']

In [86]:
X_train.isna().sum()

20150720_N    0
20150602_N    0
20150517_N    0
20150501_N    0
20150415_N    0
20150330_N    0
20150314_N    0
20150226_N    0
20150210_N    0
20150125_N    0
20150109_N    0
20141117_N    0
20141101_N    0
20141016_N    0
20140930_N    0
20140813_N    0
20140626_N    0
20140610_N    0
20140525_N    0
20140509_N    0
20140423_N    0
20140407_N    0
20140322_N    0
20140218_N    0
20140202_N    0
20140117_N    0
20140101_N    0
dtype: int64

In [87]:
y_train.isna().sum()

0

In [88]:
from sklearn.linear_model import LogisticRegression

In [89]:
model = LogisticRegression()

In [90]:
model.fit(X_train, y_train)

D:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [91]:
y_pred = model.predict(X_test)
y_pred

array(['orchard', 'forest', 'orchard', ..., 'water', 'water', 'grass'],
      dtype=object)

In [92]:
df_pred = pd.DataFrame(y_pred, columns=['class'])

# Set index to start from 1
df_pred.index = range(1, len(df_pred) + 1)

# Save to CSV
df_pred.to_csv('output.csv', index_label='ID')  # optional: index column name